# Load Modules

In [2]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
from IPython.display import display
from tqdm import tqdm

# Load Source list files

In [3]:
src_list = pd.read_csv('../source_list/pulsar_segmented_list.txt' , header=None , names = ['fname'] , na_values='NaN')['fname'].to_list()
src_list
source = pd.DataFrame()
for s in src_list[:]:
    temp = pd.read_csv('../source_list/pulsar_updated/'+s).iloc[: , 1:-1]
    source = source.append(temp)
source = source[source['num_obs']>0]
source = source.replace(np.nan , np.nan).sort_values('num_obs' , ascending=False).reset_index(drop=True)
display(source)


,NAME,PSRJ,RAJ,DECJ,RAJD,num_obs,DECJD,BINCOMP,ASSOC_MOD,ASSOC,SURVEY,PSR
0,J0024-7204W,J0024-7204W,00:24:06.0,-72:04:49.0,6.025242,543,-72.080302,MS[egc+02],GC,"GC:47Tuc,XRS:[bgh+06]",pksgc,NaN
1,J0024-7204aa,J0024-7204aa,00:24:05.6,-72:04:52.6,6.023625,534,-72.081283,NaN,GC,"GC:47Tuc,XRS:[bhc+17]",misc,NaN
2,J0024-7204V,J0024-7204V,00:24:05.6,-72:04:52.6,6.023625,534,-72.081283,NaN,GC,GC:47Tuc,pksgc,NaN
3,J0024-7204Z,J0024-7204Z,00:24:06.0,-72:05:01.4,6.025171,483,-72.083744,NaN,GC,"GC:47Tuc,XRS:[bhc+17]",misc,NaN
4,J0024-7204O,J0024-7204O,00:24:04.6,-72:04:53.7,6.019386,466,-72.081602,UL[fck+03],GC,"GC:47Tuc,XRS:[bgh+06]",pksgc,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
260,J2017+3625,J2017+3625,20:17:55.8,+36:25:07.9,304.482667,1,36.418861,NaN,GRS,"GRS:4FGL_J2017.9+3625[fermi21],XRS:[wcp+",FermiBlind,NRAD
261,J2017+0603,J2017+0603,20:17:22.7,+06:03:05.5,304.344605,1,6.051546,He[cgj+11],GRS,"GRS:4FGL_J2017.4+0602[fermi21],XRS:[pb15",misc,HE
262,J2012-2029,J2012-2029,20:12:46.6,-20:29:31,303.194167,1,-20.491944,NaN,NaN,NaN,gb350,NaN
263,J1810+1744,J1810+1744,18:10:37.2,+17:44:37.3,272.655333,1,17.743717,UL[hrm+11],GRS,"GRS:4FGL_J1810.5+1744[fermi21],OPT:[bvr+",FermiAssoc,HE


# Read Chandra data files 

In [102]:
ch_names = []
ch_ra = []
ch_dec = []
offset = []
src_id_list = [] 
n_obs_list = []
df_combined = pd.DataFrame()
for index in tqdm(range(len(source))):
    s = source['NAME'][index]
    df = pd.read_csv('../data/all_data/PULSAR_v2_all/'+s+'.csv' , index_col='index').sort_values('sepn').reset_index(drop=True)
    #display(df)
    top_match = df.loc[0]
    # Select only the source , which has highest cross-matching 
    # And create an array with this seperation value to be stored in original source list table
    # such that source list table have chandra source name , number of actual obs  and seperation.
    # Select all the obs of this cross match and append in all data file

    # Put actual source name , and give source id and obs id in this data
    df_select = df[df['name']==top_match['name']].reset_index(drop=True)
    n_obs_cur = len(df_select)
    obs_id = ['PLSR_'+str(index)+'_obs_'+str(i) for i in range(len(df_select))]
    src_id = ['PLSR_'+str(index)]*len(df_select)
    df_select.insert(0 ,'src_id' , src_id)
    df_select.insert(1 ,'obs_id' , obs_id)
    df_select.insert(2, 'num_obs' , [str(len(df_select))]*len(df_select))
    top_row = df_select.iloc[0]
    ch_names.append(top_row['name'])
    ch_ra.append(top_row['ra'])
    ch_dec.append(top_row['dec'])
    offset.append(top_row['sepn'])
    n_obs_list.append(len(df_select))
    src_id_list.append('PLSR_'+str(index))
    #display(df_select)
    df_combined = df_combined.append(df_select)
source_updated = source.copy().reset_index(drop=True)
source_updated.insert(7 , 'chandra_name' , ch_names)
source_updated.insert(8 , 'ch_ra' , ch_ra)
source_updated.insert(9 , 'ch_dec' , ch_dec)
source_updated.insert(10 , 'xoffset' , offset)
source_updated.insert(11 , 'num_obs_used' , n_obs_list)
display(source_updated)
#display(df_combined)
df_combined = df_combined.reset_index(drop=True)
df_combined = df_combined.set_index('obs_id')
df_combined

100%|██████████| 265/265 [00:21<00:00, 12.42it/s]


,NAME,PSRJ,RAJ,DECJ,RAJD,num_obs,DECJD,chandra_name,ch_ra,ch_dec,xoffset,num_obs_used,BINCOMP,ASSOC_MOD,ASSOC,SURVEY,PSR
0,J0024-7204W,J0024-7204W,00:24:06.0,-72:04:49.0,6.025242,543,-72.080302,2CXO J002406.0-720449,6.025354,-72.080320,0.419328,13,MS[egc+02],GC,"GC:47Tuc,XRS:[bgh+06]",pksgc,NaN
1,J0024-7204aa,J0024-7204aa,00:24:05.6,-72:04:52.6,6.023625,534,-72.081283,2CXO J002405.7-720452,6.023896,-72.081270,0.624119,8,NaN,GC,"GC:47Tuc,XRS:[bhc+17]",misc,NaN
2,J0024-7204V,J0024-7204V,00:24:05.6,-72:04:52.6,6.023625,534,-72.081283,2CXO J002405.7-720452,6.023896,-72.081270,0.624119,8,NaN,GC,GC:47Tuc,pksgc,NaN
3,J0024-7204Z,J0024-7204Z,00:24:06.0,-72:05:01.4,6.025171,483,-72.083744,2CXO J002406.0-720501,6.025250,-72.083770,0.318854,13,NaN,GC,"GC:47Tuc,XRS:[bhc+17]",misc,NaN
4,J0024-7204O,J0024-7204O,00:24:04.6,-72:04:53.7,6.019386,466,-72.081602,2CXO J002404.6-720453,6.019470,-72.081620,0.364113,13,UL[fck+03],GC,"GC:47Tuc,XRS:[bgh+06]",pksgc,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,J2017+3625,J2017+3625,20:17:55.8,+36:25:07.9,304.482667,1,36.418861,2CXO J201755.8+362507,304.482600,36.418820,0.290645,1,NaN,GRS,"GRS:4FGL_J2017.9+3625[fermi21],XRS:[wcp+",FermiBlind,NRAD
261,J2017+0603,J2017+0603,20:17:22.7,+06:03:05.5,304.344605,1,6.051546,2CXO J201722.6+060305,304.344500,6.051485,0.240605,1,He[cgj+11],GRS,"GRS:4FGL_J2017.4+0602[fermi21],XRS:[pb15",misc,HE
262,J2012-2029,J2012-2029,20:12:46.6,-20:29:31,303.194167,1,-20.491944,2CXO J172323.1-283757,260.846600,-28.632640,1.305641,1,NaN,NaN,NaN,gb350,NaN
263,J1810+1744,J1810+1744,18:10:37.2,+17:44:37.3,272.655333,1,17.743717,2CXO J181037.3+174437,272.655500,17.743710,1.596597,1,UL[hrm+11],GRS,"GRS:4FGL_J1810.5+1744[fermi21],OPT:[bvr+",FermiAssoc,HE


,src_id,num_obs,sepn,instrument,detect_stack_id,name,ra,dec,obsid,obi,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
obs_id,,,,,,,,,,,,,,,,,,,,,
PLSR_0_obs_0,PLSR_0,13,0.419328,ACIS,acisfJ0024178m720500_001,2CXO J002406.0-720449,6.025354,-72.080320,3386,0,...,1.328e-06,3.0,0.438,6.257e-06,13,0,0,0,0,0
PLSR_0_obs_1,PLSR_0,13,0.419328,ACIS,acisfJ0024178m720500_001,2CXO J002406.0-720449,6.025354,-72.080320,3387,0,...,1.328e-06,3.0,0.438,6.257e-06,13,0,0,0,0,0
PLSR_0_obs_2,PLSR_0,13,0.419328,ACIS,acisfJ0024178m720500_001,2CXO J002406.0-720449,6.025354,-72.080320,3385,0,...,1.328e-06,3.0,0.438,6.257e-06,13,0,0,0,0,0
PLSR_0_obs_3,PLSR_0,13,0.419328,ACIS,acisfJ0024178m720500_001,2CXO J002406.0-720449,6.025354,-72.080320,3384,0,...,1.328e-06,3.0,0.438,6.257e-06,13,0,0,0,0,0
PLSR_0_obs_4,PLSR_0,13,0.419328,ACIS,acisfJ0024178m720500_001,2CXO J002406.0-720449,6.025354,-72.080320,2738,0,...,1.328e-06,3.0,0.438,6.257e-06,13,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PLSR_260_obs_0,PLSR_260,1,0.290645,ACIS,acisfJ2018050p362653_001,2CXO J201755.8+362507,304.482600,36.418820,14699,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
PLSR_261_obs_0,PLSR_261,1,0.240605,ACIS,acisfJ2017218p060253_001,2CXO J201722.6+060305,304.344500,6.051485,12537,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
PLSR_262_obs_0,PLSR_262,1,1.305641,ACIS,acisfJ1723239m283801_001,2CXO J172323.1-283757,260.846600,-28.632640,13713,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0


## Save raw table

In [3]:
df_combined.replace(np.nan , '')
df_combined.to_csv('pulsar_raw_data.csv' ,)

NameError: name 'df_combined' is not defined

## Filter flags

In [4]:
col_to_drop = [
    'instrument',
 'detect_stack_id',
 'obsid',
 'obi',
 'src_rate_aper_m',
 'src_rate_aper_lolim_m',
 'src_rate_aper_hilim_m',
 'src_cnts_aper_u',
 'src_cnts_aper_lolim_u',
 'src_cnts_aper_hilim_u',
 'src_cnts_aper_h',
 'src_cnts_aper_lolim_h',
 'src_cnts_aper_hilim_h',
 'sat_src_flag',
 'src_rate_aper_s',
 'src_rate_aper_lolim_s',
 'src_rate_aper_hilim_s',
 'gti_start',
 'gti_stop',
 'gti_obs',
 'gti_end',
 'gti_mjd_obs',
 'mjd_ref',
 'src_rate_aper_b',
 'src_rate_aper_lolim_b',
 'src_rate_aper_hilim_b',
 'theta',
 'phi',
 'region_id',
 'src_cnts_aper_m',
 'src_cnts_aper_lolim_m',
 'src_cnts_aper_hilim_m',
 'src_cnts_aper_s',
 'src_cnts_aper_lolim_s',
 'src_cnts_aper_hilim_s',
 'src_rate_aper_h',
 'src_rate_aper_lolim_h',
 'src_rate_aper_u',
 'src_rate_aper_lolim_u',
 'src_rate_aper_hilim_u',
 'src_cnts_aper_b',
 'src_cnts_aper_lolim_b',
 'src_cnts_aper_hilim_b',
 'apec_kt',
 'apec_kt_lolim',
 'apec_kt_hilim',
 'apec_kt_rhat',
 'apec_abund',
 'apec_abund_lolim',
 'apec_abund_hilim',
 'apec_abund_rhat',
 'apec_z',
 'apec_z_lolim',
 'apec_z_hilim',
 'apec_z_rhat',
 'apec_nh',
 'apec_nh_lolim',
 'apec_nh_hilim',
 'apec_nh_rhat',
 'apec_norm',
 'apec_norm_lolim',
 'apec_norm_hilim',
 'apec_norm_rhat',
 'apec_stat',
 'detector',
 'match_type',
 'cnts_aper_b',
 'cnts_aperbkg_b',
 'cnts_aper_h',
 'cnts_aperbkg_h',
 'cnts_aper_m',
 'cnts_aperbkg_m',
 'cnts_aper_s',
 'cnts_aperbkg_s',
 'cnts_aper_u',
 'cnts_aperbkg_u',
 'pileup_flag',
 'mstr_sat_src_flag',
 'mstr_streak_src_flag',
 'mstr_flux_aper_b',
 'mstr_flux_aper_lolim_b',
 'mstr_flux_aper_hilim_b',
 'acis_num',
 'acis_hetg_num',
 'acis_letg_num',
 'hrc_num',
 'hrc_hetg_num',
 'hrc_letg_num'
]

In [9]:
df = pd.read_csv('pulsar_raw_data.csv' , index_col='obs_id')
filter_flags = [
    'pileup_flag',
    'mstr_sat_src_flag',
    'mstr_streak_src_flag',
    'sat_src_flag',
    'streak_src_flag',
    ]
df = df.replace({
    '  NAN' : np.nan , 
    '      NaN' : np.nan , 
    'FALSE' : False , 
    'False' : False , 
    ' TRUE' : True , 
    
    })
print('BEFORE FILTERING')
display(df)
for f in filter_flags:
    df = df[df[f]==False]
print('AFTER FILTERING')
display(df)
df = df.drop(columns=col_to_drop)
df

BEFORE FILTERING


,src_id,num_obs,sepn,instrument,detect_stack_id,name,ra,dec,obsid,obi,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
obs_id,,,,,,,,,,,,,,,,,,,,,
PLSR_0_obs_0,PLSR_0,13,0.419328,ACIS,acisfJ0024178m720500_001,2CXO J002406.0-720449,6.025354,-72.080320,3386,0,...,1.328e-06,3.0,0.438,6.257e-06,13,0,0,0,0,0
PLSR_0_obs_1,PLSR_0,13,0.419328,ACIS,acisfJ0024178m720500_001,2CXO J002406.0-720449,6.025354,-72.080320,3387,0,...,1.328e-06,3.0,0.438,6.257e-06,13,0,0,0,0,0
PLSR_0_obs_2,PLSR_0,13,0.419328,ACIS,acisfJ0024178m720500_001,2CXO J002406.0-720449,6.025354,-72.080320,3385,0,...,1.328e-06,3.0,0.438,6.257e-06,13,0,0,0,0,0
PLSR_0_obs_3,PLSR_0,13,0.419328,ACIS,acisfJ0024178m720500_001,2CXO J002406.0-720449,6.025354,-72.080320,3384,0,...,1.328e-06,3.0,0.438,6.257e-06,13,0,0,0,0,0
PLSR_0_obs_4,PLSR_0,13,0.419328,ACIS,acisfJ0024178m720500_001,2CXO J002406.0-720449,6.025354,-72.080320,2738,0,...,1.328e-06,3.0,0.438,6.257e-06,13,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PLSR_260_obs_0,PLSR_260,1,0.290645,ACIS,acisfJ2018050p362653_001,2CXO J201755.8+362507,304.482600,36.418820,14699,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
PLSR_261_obs_0,PLSR_261,1,0.240605,ACIS,acisfJ2017218p060253_001,2CXO J201722.6+060305,304.344500,6.051485,12537,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
PLSR_262_obs_0,PLSR_262,1,1.305641,ACIS,acisfJ1723239m283801_001,2CXO J172323.1-283757,260.846600,-28.632640,13713,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0


AFTER FILTERING


,src_id,num_obs,sepn,instrument,detect_stack_id,name,ra,dec,obsid,obi,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
obs_id,,,,,,,,,,,,,,,,,,,,,
PLSR_0_obs_0,PLSR_0,13,0.419328,ACIS,acisfJ0024178m720500_001,2CXO J002406.0-720449,6.025354,-72.080320,3386,0,...,1.328e-06,3.0,0.438,6.257e-06,13,0,0,0,0,0
PLSR_0_obs_1,PLSR_0,13,0.419328,ACIS,acisfJ0024178m720500_001,2CXO J002406.0-720449,6.025354,-72.080320,3387,0,...,1.328e-06,3.0,0.438,6.257e-06,13,0,0,0,0,0
PLSR_0_obs_2,PLSR_0,13,0.419328,ACIS,acisfJ0024178m720500_001,2CXO J002406.0-720449,6.025354,-72.080320,3385,0,...,1.328e-06,3.0,0.438,6.257e-06,13,0,0,0,0,0
PLSR_0_obs_3,PLSR_0,13,0.419328,ACIS,acisfJ0024178m720500_001,2CXO J002406.0-720449,6.025354,-72.080320,3384,0,...,1.328e-06,3.0,0.438,6.257e-06,13,0,0,0,0,0
PLSR_0_obs_4,PLSR_0,13,0.419328,ACIS,acisfJ0024178m720500_001,2CXO J002406.0-720449,6.025354,-72.080320,2738,0,...,1.328e-06,3.0,0.438,6.257e-06,13,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PLSR_259_obs_0,PLSR_259,1,1.037917,ACIS,acisfJ2028206p333210_001,2CXO J202819.8+333204,307.082800,33.534450,14826,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
PLSR_260_obs_0,PLSR_260,1,0.290645,ACIS,acisfJ2018050p362653_001,2CXO J201755.8+362507,304.482600,36.418820,14699,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
PLSR_261_obs_0,PLSR_261,1,0.240605,ACIS,acisfJ2017218p060253_001,2CXO J201722.6+060305,304.344500,6.051485,12537,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0


,src_id,num_obs,sepn,name,ra,dec,photflux_aper_s,photflux_aper_lolim_s,photflux_aper_hilim_s,flux_aper_s,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
PLSR_0_obs_0,PLSR_0,13,0.419328,2CXO J002406.0-720449,6.025354,-72.080320,2.395e-06,1.268e-06,3.522e-06,4.071e-15,...,3.671e-07,3.0,0.496,1.741e-07,5.0,0.710,1.328e-06,3.0,0.438,6.257e-06
PLSR_0_obs_1,PLSR_0,13,0.419328,2CXO J002406.0-720449,6.025354,-72.080320,0.000e+00,0.000e+00,6.265e-07,0.000e+00,...,3.671e-07,3.0,0.496,1.741e-07,5.0,0.710,1.328e-06,3.0,0.438,6.257e-06
PLSR_0_obs_2,PLSR_0,13,0.419328,2CXO J002406.0-720449,6.025354,-72.080320,0.000e+00,0.000e+00,1.994e-06,0.000e+00,...,3.671e-07,3.0,0.496,1.741e-07,5.0,0.710,1.328e-06,3.0,0.438,6.257e-06
PLSR_0_obs_3,PLSR_0,13,0.419328,2CXO J002406.0-720449,6.025354,-72.080320,1.484e-06,5.832e-07,2.333e-06,2.669e-15,...,3.671e-07,3.0,0.496,1.741e-07,5.0,0.710,1.328e-06,3.0,0.438,6.257e-06
PLSR_0_obs_4,PLSR_0,13,0.419328,2CXO J002406.0-720449,6.025354,-72.080320,1.214e-06,9.960e-07,1.433e-06,2.118e-15,...,3.671e-07,3.0,0.496,1.741e-07,5.0,0.710,1.328e-06,3.0,0.438,6.257e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PLSR_259_obs_0,PLSR_259,1,1.037917,2CXO J202819.8+333204,307.082800,33.534450,3.7770000000000005e-07,1.488e-07,6.065999999999999e-07,5.601e-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PLSR_260_obs_0,PLSR_260,1,0.290645,2CXO J201755.8+362507,304.482600,36.418820,1.147e-06,2.73e-07,2.02e-06,1.689e-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PLSR_261_obs_0,PLSR_261,1,0.240605,2CXO J201722.6+060305,304.344500,6.051485,2.768e-06,1.7909999999999999e-06,3.6910000000000003e-06,5.882e-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
arranged_cols = [
    "src_id",
    "num_obs",
    "sepn",
    "name",
    "ra",
    "dec",
    "livetime",
    "significance",
    "likelihood",
    "flux_significance_b",
    "flux_significance_h",
    "flux_significance_m",
    "flux_significance_s",
    "flux_significance_u",
    "photflux_aper_s",
    "photflux_aper_lolim_s",
    "photflux_aper_hilim_s",
    "flux_aper_s",
    "flux_aper_lolim_s",
    "flux_aper_hilim_s",
    "photflux_aper_b",
    "photflux_aper_lolim_b",
    "photflux_aper_hilim_b",
    "flux_aper_h",
    "flux_aper_lolim_h",
    "flux_aper_hilim_h",
    "photflux_aper_u",
    "photflux_aper_lolim_u",
    "photflux_aper_hilim_u",
    "flux_aper_b",
    "flux_aper_lolim_b",
    "flux_aper_hilim_b",
    "flux_aper_u",
    "flux_aper_lolim_u",
    "flux_aper_hilim_u",
    "flux_aper_m",
    "flux_aper_lolim_m",
    "flux_aper_hilim_m",
    "photflux_aper_h",
    "photflux_aper_lolim_h",
    "photflux_aper_hilim_h",
    "photflux_aper_m",
    "photflux_aper_lolim_m",
    "photflux_aper_hilim_m",
    "hard_hm",
    "hard_hm_lolim",
    "hard_hm_hilim",
    "hard_hs",
    "hard_hs_lolim",
    "hard_hs_hilim",
    "hard_ms",
    "hard_ms_lolim",
    "hard_ms_hilim",
    "flux_powlaw",
    "flux_powlaw_lolim",
    "flux_powlaw_hilim",
    "powlaw_gamma",
    "powlaw_gamma_lolim",
    "powlaw_gamma_hilim",
    "powlaw_gamma_rhat",
    "powlaw_nh",
    "powlaw_nh_lolim",
    "powlaw_nh_hilim",
    "powlaw_nh_rhat",
    "powlaw_ampl",
    "powlaw_ampl_lolim",
    "powlaw_ampl_hilim",
    "powlaw_ampl_rhat",
    "powlaw_stat",
    "flux_bb",
    "flux_bb_lolim",
    "flux_bb_hilim",
    "bb_kt",
    "bb_kt_lolim",
    "bb_kt_hilim",
    "bb_kt_rhat",
    "bb_nh",
    "bb_nh_lolim",
    "bb_nh_hilim",
    "bb_nh_rhat",
    "bb_ampl",
    "bb_ampl_lolim",
    "bb_ampl_hilim",
    "bb_ampl_rhat",
    "bb_stat",
    "flux_brems",
    "flux_brems_lolim",
    "flux_brems_hilim",
    "brems_kt",
    "brems_kt_lolim",
    "brems_kt_hilim",
    "brems_kt_rhat",
    "brems_nh",
    "brems_nh_lolim",
    "brems_nh_hilim",
    "brems_nh_rhat",
    "brems_norm",
    "brems_norm_lolim",
    "brems_norm_hilim",
    "brems_norm_rhat",
    "brems_stat",
    "flux_apec",
    "flux_apec_lolim",
    "flux_apec_hilim",
    "var_index_b",
    "var_prob_b",
    "ks_prob_b",
    "kp_prob_b",
    "var_sigma_b",
    "var_mean_b",
    "var_min_b",
    "var_max_b",
    "var_index_h",
    "var_prob_h",
    "ks_prob_h",
    "kp_prob_h",
    "var_sigma_h",
    "var_mean_h",
    "var_min_h",
    "var_max_h",
    "var_index_m",
    "var_prob_m",
    "ks_prob_m",
    "kp_prob_m",
    "var_sigma_m",
    "var_mean_m",
    "var_min_m",
    "var_max_m",
    "var_index_s",
    "var_prob_s",
    "ks_prob_s",
    "kp_prob_s",
    "var_sigma_s",
    "var_mean_s",
    "var_min_s",
    "var_max_s",
    "var_index_u",
    "var_prob_u",
    "ks_prob_u",
    "kp_prob_u",
    "var_sigma_u",
    "var_mean_u",
    "var_min_u",
    "var_max_u",
    "var_inter_index_b",
    "var_inter_prob_b",
    "var_inter_sigma_b",
    "var_inter_index_h",
    "var_inter_prob_h",
    "var_inter_sigma_h",
    "var_inter_index_m",
    "var_inter_prob_m",
    "var_inter_sigma_m",
    "var_inter_index_s",
    "var_inter_prob_s",
    "var_inter_sigma_s",
    "var_inter_index_u",
    "var_inter_prob_u",
    "var_inter_sigma_u",
]

id_cols = [
   "src_id",
    "num_obs",
    "sepn",
    "name",
    "ra",
    "dec",
    "livetime",
    "significance",
    "likelihood",
]
val_cols = [
    
]

In [11]:
df_id = df[id_cols]
df_val = df.drop(columns=id_cols)
val_col_list = df_val.columns.to_list()
for v in val_col_list:
    df_val.loc[:,v] = pd.to_numeric(df_val.loc[:,v])
df_val

,photflux_aper_s,photflux_aper_lolim_s,photflux_aper_hilim_s,flux_aper_s,flux_aper_lolim_s,flux_aper_hilim_s,streak_src_flag,likelihood_b,likelihood_h,likelihood_m,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
PLSR_0_obs_0,2.395000e-06,1.268000e-06,3.522000e-06,4.071000e-15,2.155000e-15,5.987000e-15,False,43.88,NaN,13.38,...,3.671000e-07,3.0,0.496,1.741000e-07,5.0,0.71,0.000001,3.0,0.438,0.000006
PLSR_0_obs_1,0.000000e+00,0.000000e+00,6.265000e-07,0.000000e+00,0.000000e+00,8.865000e-16,False,NaN,NaN,NaN,...,3.671000e-07,3.0,0.496,1.741000e-07,5.0,0.71,0.000001,3.0,0.438,0.000006
PLSR_0_obs_2,0.000000e+00,0.000000e+00,1.994000e-06,0.000000e+00,0.000000e+00,2.836000e-15,False,10.80,NaN,NaN,...,3.671000e-07,3.0,0.496,1.741000e-07,5.0,0.71,0.000001,3.0,0.438,0.000006
PLSR_0_obs_3,1.484000e-06,5.832000e-07,2.333000e-06,2.669000e-15,1.013000e-15,4.326000e-15,False,15.21,NaN,NaN,...,3.671000e-07,3.0,0.496,1.741000e-07,5.0,0.71,0.000001,3.0,0.438,0.000006
PLSR_0_obs_4,1.214000e-06,9.960000e-07,1.433000e-06,2.118000e-15,1.737000e-15,2.498000e-15,False,196.94,54.31,20.63,...,3.671000e-07,3.0,0.496,1.741000e-07,5.0,0.71,0.000001,3.0,0.438,0.000006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PLSR_259_obs_0,3.777000e-07,1.488000e-07,6.066000e-07,5.601000e-16,2.206000e-16,8.995000e-16,False,31.37,16.81,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PLSR_260_obs_0,1.147000e-06,2.730000e-07,2.020000e-06,1.689000e-15,4.020000e-16,2.975000e-15,False,33.08,12.08,15.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PLSR_261_obs_0,2.768000e-06,1.791000e-06,3.691000e-06,5.882000e-15,3.806000e-15,7.843000e-15,False,107.22,NaN,154.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_ok = pd.concat([df_id , df_val] , axis=1)
df_ok

,src_id,num_obs,sepn,name,ra,dec,livetime,significance,likelihood,photflux_aper_s,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
PLSR_0_obs_0,PLSR_0,13,0.419328,2CXO J002406.0-720449,6.025354,-72.080320,5544.8,13.73,756.13850,2.395000e-06,...,3.671000e-07,3.0,0.496,1.741000e-07,5.0,0.71,0.000001,3.0,0.438,0.000006
PLSR_0_obs_1,PLSR_0,13,0.419328,2CXO J002406.0-720449,6.025354,-72.080320,5734.6,13.73,756.13850,0.000000e+00,...,3.671000e-07,3.0,0.496,1.741000e-07,5.0,0.71,0.000001,3.0,0.438,0.000006
PLSR_0_obs_2,PLSR_0,13,0.419328,2CXO J002406.0-720449,6.025354,-72.080320,1928.3,13.73,756.13850,0.000000e+00,...,3.671000e-07,3.0,0.496,1.741000e-07,5.0,0.71,0.000001,3.0,0.438,0.000006
PLSR_0_obs_3,PLSR_0,13,0.419328,2CXO J002406.0-720449,6.025354,-72.080320,5307.0,13.73,756.13850,1.484000e-06,...,3.671000e-07,3.0,0.496,1.741000e-07,5.0,0.71,0.000001,3.0,0.438,0.000006
PLSR_0_obs_4,PLSR_0,13,0.419328,2CXO J002406.0-720449,6.025354,-72.080320,68771.4,13.73,756.13850,1.214000e-06,...,3.671000e-07,3.0,0.496,1.741000e-07,5.0,0.71,0.000001,3.0,0.438,0.000006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PLSR_259_obs_0,PLSR_259,1,1.037917,2CXO J202819.8+333204,307.082800,33.534450,24736.0,2.97,31.36590,3.777000e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PLSR_260_obs_0,PLSR_260,1,0.290645,2CXO J201755.8+362507,304.482600,36.418820,9843.9,3.03,33.08191,1.147000e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PLSR_261_obs_0,PLSR_261,1,0.240605,2CXO J201722.6+060305,304.344500,6.051485,9906.6,3.82,154.50270,2.768000e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_ok.to_csv('pulsar_data.csv')